# AlphaZero Training on Knight Capture Game

This notebook runs the AlphaZero training pipeline.


In [ ]:
import torch
from model import AlphaZeroNet
from train import AlphaZeroTrainer, ReplayBuffer
from config import Config


In [ ]:
# Configuration
checkpoint_path = None  # Set to a path like 'checkpoints/model_iter_1.pth' to resume training

config = Config()


In [ ]:
# Set device
device = torch.device(config.device if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


In [ ]:
# Initialize model
model = AlphaZeroNet(
    board_size=config.board_size,
    num_channels=config.num_channels,
    num_residual_blocks=config.num_residual_blocks
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
# Initialize trainer
trainer = AlphaZeroTrainer(
    model=model,
    num_simulations=config.num_simulations,
    num_games=config.num_games,
    num_iterations=config.num_iterations,
    num_epochs=config.num_epochs,
    num_workers=config.num_workers,
    batch_size=config.batch_size,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    c_puct=config.c_puct,
    temperature=config.temperature,
    device=str(device),
    save_dir=config.save_dir
)

# Set max game length from config
trainer.max_game_length = config.max_game_length

# Update replay buffer size
trainer.replay_buffer = ReplayBuffer(max_size=config.buffer_size)


In [ ]:
# Load checkpoint if provided
if checkpoint_path:
    trainer.load_checkpoint(checkpoint_path)
    print(f"Loaded checkpoint from {checkpoint_path}")
else:
    print("Starting training from scratch")


In [ ]:
# Start training
trainer.train()

print("\nTraining completed!")
